<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Uncapacitated Facility Location - Problem #01



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)



<br>
<br>


<a id='prob'></a>
## Problem #01

<br>
Facilities <br>
$|I| = 9$ 
<br>
<br>
Clients <br>
$|J| = 2$ 

<br>
<br>
$\alpha = 1$ (cost per mile of driving)


| <i></i> | Coordinates |  
| --- | --- | 
| Client 1 | (0,1.5) |
| Client 2 | (2.5,1.2) |



| <i></i> | coordinates | fixed cost |
| --- | --- |  --- |
| Warehouse 1 | (0,0) | 3 |
| Warehouse 2 | (0,1) | 2 |
| Warehouse 3 | (0,2) | 3 |
| Warehouse 4 | (1,0) | 1 |
| Warehouse 5 | (1,1) | 3 | 
| Warehouse 6 | (1,2) | 3 |
| Warehouse 7 | (2,0) | 4 |
| Warehouse 8 | (2,1) | 3 |  
| Warehouse 9 | (2,2) | 2 |


<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [17]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB


##### Sets and Indices #####
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
num_facilities = len(facilities)
num_customers = len(customers)

##### Parameters #####
cost_per_mile = 1

setup_cost = [3,2,3,1,3,3,4,3,2]
# Euclidean distance between a facility and customer sites
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}


# MIP  model formulation
m = gp.Model('UFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

##### Objective Function #####
m.setObjective(x.prod(setup_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0x0939f503
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 4.72371 

Optimal solution foun

In [18]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

m.write('UFLP_01_Simplex.lp')


Build a warehouse at location 4.

Client 1 receives 100.0 % of its demand  from Warehouse 4 .

Client 2 receives 100.0 % of its demand  from Warehouse 4 .

Optimal total: 4.72371290896185


<br>
<br>

<a id='random'></a>
### Random Heuristic 





In [19]:
#very naive/simple
#for each client, choose randomly one of the Facilities available
import random

result=[]
i=0
#choose the Facility for each customer
for w in customers:
    result.append((i,random.randint(0, len(facilities)-1)))
    i+=1

#remove duplication of facilities in order to print properly
facs=[]
for i in result:
    facs.append(i[1])
facs=list(set(facs))
    
print(result)

#calculate the setup_cost
totalC=0
for w in facs:
    totalC += setup_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in result:
    totalC += cost_per_mile*compute_distance(customers[i[0]], facilities[i[1]]) 
    
print(totalC)



[(0, 3), (1, 1)]
3
7.310762878528886


In [20]:
for i in facs:
    print(f"\nBuild a warehouse at location {i + 1}.")

for i in result:
    print(f"\nClient {i[0] + 1} receives 100% of its demand  from Warehouse {i[1] + 1} .")

print(f"\nOptimal total:", totalC)


Build a warehouse at location 2.

Build a warehouse at location 4.

Client 1 receives 100% of its demand  from Warehouse 4 .

Client 2 receives 100% of its demand  from Warehouse 2 .

Optimal total: 7.310762878528886


<a id='lowShip'></a>
### Lowest Shipping Cost (per client) Heuristic



In [21]:
#Get the lowest shipping cost for each Client


#choose the lowest cost 
path={}
for i in enumerate(customers):
    aux={}
    aux_key=()
    for j in enumerate(facilities):

        #empty list for the Client
        if not aux:
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) 
            aux_key = ((i[0],j[0])) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>shipping_cost.get((i[0],j[0])):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0]))             
            aux_key = ((i[0],j[0]))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
facs=list(set(facs))
#print (facs)

totalC=0
for w in facs:
    totalC += setup_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    print(i)
    totalC += i 
    
print(totalC)


{(0, 1): 0.5, (1, 7): 0.5385164807134504}
5
0.5
0.5385164807134504
6.038516480713451


<a id='maxShip'></a>
### Greatest Shipping Cost (per client) Heuristic



In [22]:
#Get the greatest shipping cost for each Client


#choose the lowest cost 
path={}
for i in enumerate(customers):
    aux={}
    aux_key=()
    for j in enumerate(facilities):

        #empty list for the Client
        if not aux:
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) 
            aux_key = ((i[0],j[0])) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>shipping_cost.get((i[0],j[0])):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0]))             
            aux_key = ((i[0],j[0]))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
facs=list(set(facs))
#print (facs)

totalC=0
for w in facs:
    totalC += setup_cost[w]
    #print(w)
print(facs)
print(totalC)

#calculate the shipping cost
for i in path.values():
    print(i)
    totalC += i 
    
print(totalC)


{(0, 6): 2.5, (1, 0): 2.773084924772409}
[0, 6]
7
2.5
2.773084924772409
12.273084924772409


<a id='lowShipFix'></a>
### Lowest Shipping & Fixed Costs Heuristic



In [23]:
#Get the lowest shipping cost and lowest fixed cost 


#choose the lowest cost 
path={}
for i in enumerate(customers):
    aux={}
    aux_key=()
    for j in enumerate(facilities):

        #empty list for the Client
        if not aux:
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) + setup_cost[j[0]]
            aux_key = ((i[0],j[0])) 
            #print(aux)
            #print(aux_key)
        elif aux[aux_key]>(shipping_cost.get((i[0],j[0]))+ setup_cost[j[0]]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) + setup_cost[j[0]]            
            aux_key = ((i[0],j[0]))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
facs=list(set(facs))
print ("facs: ",facs)

totalC=0
for w in facs:
    totalC += setup_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    print(i)
    totalC += i 
    
print(totalC)


{(0, 1): 2.5, (1, 3): 2.920937271229855}
facs:  [1, 3]
3
2.5
2.920937271229855
8.420937271229855


<a id='maxShipFix'></a>
### Greatest Shipping & Fixed Costs Heuristic



In [24]:
#Get the Greatest shipping cost and greates fixed cost 


#choose the lowest cost 
path={}
for i in enumerate(customers):
    aux={}
    aux_key=()
    for j in enumerate(facilities):

        #empty list for the Client
        if not aux:
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) + setup_cost[j[0]]
            aux_key = ((i[0],j[0])) 
            #print(aux)
            #print(aux_key)
        elif not aux[aux_key]>(shipping_cost.get((i[0],j[0]))+ setup_cost[j[0]]):
            #print("....")
            #print (i[0],j[0])
            #print(aux[aux_key])
            #print(shipping_cost.get((i[0],j[0])))
            #print ("  ")
            aux.pop(aux_key)
            aux[(i[0],j[0])] = shipping_cost.get((i[0],j[0])) + setup_cost[j[0]]            
            aux_key = ((i[0],j[0]))
            
        #print(aux_key)
    #print ("xxxxxxx")
    path.update(aux)

print(path)    

#calculate the setup_cost
facs=[]
for i in path:
    facs.append(i[1])
facs=list(set(facs))
print ("facs: ",facs)

totalC=0
for w in facs:
    totalC += setup_cost[w]
    #print(w)

print(totalC)

#calculate the shipping cost
for i in path.values():
    print(i)
    totalC += i 
    
print(totalC)


{(0, 6): 6.5, (1, 0): 5.773084924772409}
facs:  [0, 6]
7
6.5
5.773084924772409
19.27308492477241


In [25]:
#customers = [(0,1.5), (2.5,1.2)]
#facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
#print (facilities)
#print (customers)

##### Parameters #####
#cost_per_mile = 1

print (setup_cost)
# shipping costs
print (shipping_cost)


[3, 2, 3, 1, 3, 3, 4, 3, 2]
{(0, 0): 1.5, (0, 1): 0.5, (0, 2): 0.5, (0, 3): 1.8027756377319946, (0, 4): 1.118033988749895, (0, 5): 1.118033988749895, (0, 6): 2.5, (0, 7): 2.0615528128088303, (0, 8): 2.0615528128088303, (1, 0): 2.773084924772409, (1, 1): 2.5079872407968904, (1, 2): 2.6248809496813377, (1, 3): 1.9209372712298547, (1, 4): 1.5132745950421556, (1, 5): 1.7, (1, 6): 1.3, (1, 7): 0.5385164807134504, (1, 8): 0.9433981132056605}
